# MOS Capacitor

## Capacitance-Voltage (C-V) Analysis

The MOS capacitor is the fundamental building block for understanding MOSFET operation. C-V measurements reveal important information about oxide quality and interface states.

**Learning Objectives:**
- Understand MOS capacitor operation regions
- Simulate C-V characteristics
- Analyze accumulation, depletion, and inversion
- Study flat-band voltage and oxide charges

---

## 1. MOS Capacitor Physics

### 1.1 Structure

```
    Metal Gate
    ==========
    |  Oxide  | (SiO2)
    ==========
    |         |
    | Silicon | (P-type or N-type)
    |         |
    ==========
    Back Contact
```

### 1.2 Operation Regions (P-type substrate)

- **Accumulation** (Vg < 0): Holes accumulate at surface
- **Flat-band** (Vg = VFB): Bands are flat
- **Depletion** (VFB < Vg < VT): Surface depleted of holes
- **Inversion** (Vg > VT): Electrons form inversion layer

### 1.3 Key Parameters

- **Oxide capacitance**: $C_{ox} = \epsilon_{ox}/t_{ox}$

- **Flat-band voltage**: $V_{FB} = \phi_{ms} - Q_f/C_{ox}$

- **Threshold voltage**: $V_T = V_{FB} + 2\phi_F + Q_d/C_{ox}$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from nanohubpadre import create_mos_capacitor

# Physical constants
q = 1.6e-19
eps_0 = 8.85e-14  # F/cm
eps_ox = 3.9 * eps_0
eps_si = 11.7 * eps_0
kT = 0.0259  # eV at 300K
ni = 1.5e10  # intrinsic carrier concentration

# MOS parameters
tox = 5e-7   # 5 nm oxide
Na = 1e17    # P-type substrate doping

Cox = eps_ox / tox
phi_F = kT * np.log(Na / ni)

print("MOS Capacitor Parameters:")
print("="*40)
print(f"Oxide thickness: {tox*1e7:.0f} nm")
print(f"Cox = {Cox*1e6:.2f} uF/cm^2")
print(f"phi_F = {phi_F:.3f} V")

---

## 2. Creating a MOS Capacitor Simulation

In [ ]:
# Create MOS capacitor
sim_moscap = create_mos_capacitor(
    # Geometry
    oxide_thickness=0.005,      # 5 nm oxide
    silicon_thickness=0.05,     # 50 nm silicon
    device_width=1.0,
    
    # Doping
    substrate_doping=1e17,      # P-type doping
    substrate_type='p',
    
    # Gate
    gate_type='n_poly',         # N+ polysilicon gate
    
    # Oxide
    oxide_permittivity=3.9,
    
    # Models
    temperature=300,
    
    # Output
    log_bands_eq=True
)

print("MOS Capacitor Configuration:")
print("="*40)
print("Oxide: SiO2, 5 nm")
print("Substrate: P-type Si, 1e17 cm^-3")
print("Gate: N+ polysilicon")

# Run the simulation
print("\nRunning equilibrium simulation...")
result = sim_moscap.run()
if result.returncode == 0:
    print("Simulation completed successfully!")
else:
    print(f"Simulation failed with return code: {result.returncode}")

In [ ]:
# View generated deck
print("PADRE Input Deck:")
print("="*60)
print(sim_moscap.generate_deck())

# Plot equilibrium band diagram
print("\n" + "="*60)
print("Equilibrium Band Diagram:")
sim_moscap.plot_band_diagram(title="MOS Capacitor - Equilibrium (Near Flat-band)")

---

## 3. C-V Characteristics

In [ ]:
# C-V simulation
sim_cv = create_mos_capacitor(
    oxide_thickness=0.005,
    silicon_thickness=0.05,
    substrate_doping=1e17,
    substrate_type='p',
    gate_type='n_poly',
    
    # Enable C-V logging
    log_cv=True,
    cv_file="cv_data",
    
    # Gate voltage sweep
    vg_sweep=(-2.0, 2.0, 0.1),  # -2V to +2V
    ac_frequency=1e6             # 1 MHz
)

print("C-V Simulation Configuration:")
print("="*40)
print("Gate voltage: -2V to +2V")
print("AC frequency: 1 MHz")

# Run the simulation
print("\nRunning C-V simulation...")
result_cv = sim_cv.run()
if result_cv.returncode == 0:
    print("Simulation completed successfully!")
else:
    print(f"Simulation failed with return code: {result_cv.returncode}")

In [ ]:
# Theoretical C-V curve and PADRE simulation comparison
def mos_cv(Vg, Na, tox, VFB=0):
    """Calculate MOS capacitor C-V curve"""
    Cox = eps_ox / tox
    phi_F = kT * np.log(Na / ni)
    
    C = np.zeros_like(Vg)
    
    for i, vg in enumerate(Vg):
        Vgb = vg - VFB
        
        if Vgb < 0:  # Accumulation
            C[i] = Cox
        elif Vgb < 2 * phi_F:  # Depletion
            # Simplified depletion capacitance
            Wd = np.sqrt(2 * eps_si * Vgb / (q * Na))
            Cd = eps_si / Wd
            C[i] = 1 / (1/Cox + 1/Cd)
        else:  # Strong inversion (high frequency)
            Wd_max = np.sqrt(4 * eps_si * phi_F / (q * Na))
            Cd_min = eps_si / Wd_max
            C[i] = 1 / (1/Cox + 1/Cd_min)  # HF: Cmin
    
    return C

Vg = np.linspace(-2, 2, 100)
C_hf = mos_cv(Vg, Na, tox)

plt.figure(figsize=(10, 6))

# Theoretical curves
plt.plot(Vg, C_hf / Cox * 100, 'b--', linewidth=2, alpha=0.5, label='Theory: High Frequency')

# Low frequency curve (inversion capacitance = Cox)
C_lf = mos_cv(Vg, Na, tox)
C_lf[Vg > 0.5] = Cox  # LF: inversion charge follows
plt.plot(Vg, C_lf / Cox * 100, 'r--', linewidth=2, alpha=0.5, label='Theory: Low Frequency')

# Try to get PADRE C-V data
try:
    cv_data = sim_cv.get_cv_data()
    if cv_data is not None:
        Vg_sim, C_sim = cv_data.get_cv_data()
        # Normalize to Cox
        Cox_val = eps_ox / (tox)
        plt.plot(Vg_sim, C_sim / Cox_val * 100, 'g-', linewidth=2, label='PADRE Simulation')
        print("PADRE C-V data extracted successfully!")
except Exception as e:
    print(f"Note: C-V data extraction may require additional output parsing: {e}")
    print("Using theoretical curves for comparison")

plt.xlabel('Gate Voltage Vg (V)', fontsize=12)
plt.ylabel('Capacitance C/Cox (%)', fontsize=12)
plt.title('MOS Capacitor C-V Characteristics', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.xlim(-2, 2)
plt.ylim(0, 110)

# Add region labels
plt.annotate('Accumulation', xy=(-1.5, 95), fontsize=10)
plt.annotate('Depletion', xy=(0, 60), fontsize=10)
plt.annotate('Inversion', xy=(1.5, 40), fontsize=10)

plt.tight_layout()
plt.show()

# Show theoretical parameters
phi_F_val = kT * np.log(Na / ni)
Wd_max = np.sqrt(4 * eps_si * phi_F_val / (q * Na)) * 1e4  # um
Cd_min = eps_si / (Wd_max * 1e-4)
Cmin_Cox = 1 / (1 + Cox / Cd_min)

print("\nTheoretical Parameters:")
print("="*40)
print(f"Surface potential at inversion: 2*phi_F = {2*phi_F_val:.3f} V")
print(f"Maximum depletion width: Wd_max = {Wd_max*1e3:.1f} nm")
print(f"Cmin/Cox = {Cmin_Cox*100:.1f}%")

---

## 4. Effect of Doping

In [ ]:
# Compare different substrate dopings
doping_levels = [1e16, 1e17, 1e18]  # cm^-3

plt.figure(figsize=(10, 6))

for Na in doping_levels:
    C = mos_cv(Vg, Na, tox)
    plt.plot(Vg, C / Cox * 100, linewidth=2, label=f'Na = {Na:.0e} cm^-3')
    
    sim = create_mos_capacitor(
        oxide_thickness=0.005,
        substrate_doping=Na,
        log_cv=True,
        vg_sweep=(-2.0, 2.0, 0.1)
    )

plt.xlabel('Gate Voltage Vg (V)', fontsize=12)
plt.ylabel('Capacitance C/Cox (%)', fontsize=12)
plt.title('Effect of Substrate Doping on C-V', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.xlim(-2, 2)

plt.tight_layout()
plt.show()

print("\nKey observation:")
print("Higher doping -> smaller depletion width -> higher Cmin")

---

## 5. Effect of Oxide Thickness

In [ ]:
# Compare different oxide thicknesses
tox_values = [0.002, 0.005, 0.010]  # 2, 5, 10 nm

print("Oxide Thickness Comparison:")
print("="*50)

for tox_um in tox_values:
    tox_cm = tox_um * 1e-4
    Cox_val = eps_ox / tox_cm
    
    sim = create_mos_capacitor(
        oxide_thickness=tox_um,
        substrate_doping=1e17,
        log_cv=True,
        vg_sweep=(-2.0, 2.0, 0.1)
    )
    
    print(f"tox = {tox_um*1000:.0f} nm: Cox = {Cox_val*1e6:.2f} uF/cm^2")

---

## 6. Band Diagrams in Different Regions

In [ ]:
# Simulate band diagrams at different gate voltages
sim_bands = create_mos_capacitor(
    oxide_thickness=0.005,
    substrate_doping=1e17,
    log_bands_eq=True,
    log_cv=True,
    vg_sweep=(-1.0, 1.5, 0.5)  # Sample points
)

print("Band diagram simulation at different Vg:")
print("  Vg = -1.0V (Accumulation)")
print("  Vg = 0.0V (Near flat-band)")
print("  Vg = 0.5V (Depletion)")
print("  Vg = 1.0V (Weak inversion)")
print("  Vg = 1.5V (Strong inversion)")

---

## 7. Complete Simulation Example

In [ ]:
# Complete MOS capacitor characterization
sim_complete = create_mos_capacitor(
    # Geometry
    oxide_thickness=0.005,
    silicon_thickness=0.05,
    device_width=1.0,
    
    # Mesh
    nx=3,
    ny_oxide=15,
    ny_silicon=35,
    
    # Doping
    substrate_doping=1e17,
    substrate_type='p',
    
    # Gate
    gate_type='n_poly',
    
    # Oxide
    oxide_permittivity=3.9,
    oxide_qf=0,  # No fixed charge
    
    # Models
    temperature=300,
    conmob=True,
    fldmob=True,
    
    # Output
    log_cv=True,
    cv_file="moscap_cv",
    log_bands_eq=True,
    
    # C-V sweep
    vg_sweep=(-2.5, 2.5, 0.05),
    ac_frequency=1e6
)

print("Complete MOS Capacitor Simulation")
print("="*50)
print(sim_complete.generate_deck())

---

## Summary

In this notebook, you learned:

1. **MOS Structure**: Metal-Oxide-Semiconductor stack
2. **Operating Regions**: Accumulation, depletion, inversion
3. **C-V Analysis**: High vs low frequency behavior
4. **Doping Effects**: Higher doping -> higher Cmin
5. **Oxide Thickness**: Thinner oxide -> higher Cox

**Key Equations:**
- $C_{ox} = \epsilon_{ox}/t_{ox}$
- $C_{min}/C_{ox}$ depends on $W_d^{max}$ and doping

**Next**: [08 - MESFET](08_MESFET.ipynb) - Metal-semiconductor FETs